<a href="https://colab.research.google.com/github/dlmf15/EFE-clipping/blob/main/EFE_Clipping_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Instalación
%%capture
!pip install --upgrade -q gspread
!pip install python-docx
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install tomorrow
!curl https://upload.wikimedia.org/wikipedia/commons/e/e1/Logo_de_la_Agencia_EFE.jpg > efe_logo.jpg
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1cayft0rvuMkIgWwpjFHlaPHgKXy-R6aS' -O efe_portada.jpg
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!wget --no-check-certificate 'https://drive.google.com/open?id=1Z6ZR70tfTeHra0LfA6wkluEkIIGotXkh' -O i_dont_care_about_cookies.crx
!pip install tweepy
!pip install jsonpickle

import tweepy
import sys
import os
import jsonpickle
from tomorrow import threads
import docx
import nltk

nltk.download('stopwords')

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

def tokenizer(text):
  
  stop_words = set(stopwords.words('spanish'))

  token = RegexpTokenizer(r'\w+')
  text = token.tokenize(text.lower())

  keywords = []
  for w in text:
    if w not in stop_words:
        keywords.append(w)
  keywords_set = set(keywords)
  return(keywords_set)


def add_hyperlink(paragraph, url, text, color):
    """
    A function that places a hyperlink within a paragraph object.

    :param paragraph: The paragraph we are adding the hyperlink to.
    :param url: A string containing the required url
    :param text: The text displayed for the url
    :return: The hyperlink object
    """

    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element
    new_run = docx.oxml.shared.OxmlElement('w:r')

    # Create a new w:rPr element
    rPr = docx.oxml.shared.OxmlElement('w:rPr')
    
    #Add color if it is given
    if not color is None:
      c = docx.oxml.shared.OxmlElement('w:color')
      c.set(docx.oxml.shared.qn('w:val'), color)
      rPr.append(c)
      
    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    paragraph._p.append(hyperlink)

    return hyperlink
  
def delete_paragraph(paragraph):
    p = paragraph._element
    p.getparent().remove(p)
    p._p = p._element = None
  
#Set up chromedriver
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse
from selenium import webdriver
import time

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--window-size=1200,900')
chrome_options.add_experimental_option("prefs", {"profile.default_content_setting_values.cookies":2})
chrome_options.add_argument('user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578 Safari/537.36')
chrome_options.add_argument('--load-extension=/content/i_dont_care_about_cookies.crx')


headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578 Safari/537.36'}


In [ ]:
#@title Input Texto

titulo = "Militao: \"Soy un jugador polivalente que juega por amor a lo que hace\"" #@param {type:"string"}
title_set = tokenizer(titulo)
print(title_set)

query = ' AND '.join(str(s) for s in title_set)

informe_de_seguimiento = "Test" #@param {type:"string"}
informe_de_seguimiento = informe_de_seguimiento.upper()

{'militao', 'jugador', 'hace', 'amor', 'juega', 'polivalente'}


In [ ]:
#@title Total paginas en Google(todo y vídeos) y Duckduckgo
# find next page, all urls at once
rlist =[] 
pages = []
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get('https://www.google.es/search?num=100&filter=0&q='+titulo+'+-filetype:pdf+-filetype:doc+-filetype:docx+-filetype:xls')
#driver.get('https://www.google.com/search?q=In%C3%A9s,+la+%C3%BAltima+cabrera,+esperanza+para+un+mundo+que+agoniza+en+Europa&filter=0&biw=1211&bih=665')

pages.append(driver.page_source)
soup = BeautifulSoup(driver.page_source, 'lxml')
find_pnnext = soup.find('a', attrs={'id':'pnnext'})

while True:
  
  if find_pnnext:
    next_page = find_pnnext.get('href')
    driver.get('https://www.google.es'+next_page)
    pages.append(driver.page_source)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    find_pnnext = soup.find('a', attrs={'id':'pnnext'})
  else:
    break
ulist = []
for p in pages:
  soup = BeautifulSoup(p, 'lxml')
  #print(soup)
  [ulist.append(u.find('a').get('href')) for u in soup.find_all('div', attrs={'class': 'r'})
           if str(u.find('a').get('href')).endswith('.pdf') == False]

driver.quit()

# search video pages
pages = []
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get('https://www.google.es/search?num=100&filter=0&tbm=vid&q='+titulo)
pages.append(driver.page_source)
soup = BeautifulSoup(driver.page_source, 'lxml')
find_pnnext = soup.find('a', attrs={'id':'pnnext'})

while True:
  
  if find_pnnext:
    next_page = find_pnnext.get('href')
    driver.get('https://www.google.es'+next_page)
    pages.append(driver.page_source)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    find_pnnext = soup.find('a', attrs={'id':'pnnext'})
  else:
    break
vlist = []
for p in pages:
  soup = BeautifulSoup(p, 'lxml')
  #print(soup)
  [vlist.append(v.find('a').get('href')) for v in soup.find_all('div', attrs={'class': 'r'})]
  
driver.quit()

#duckduckgo
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get('https://www.duckduckgo.com/?q='+titulo+'&kav=1')

for i in range(0,10):

	js="var q=document.documentElement.scrollTop=100000"
	driver.execute_script(js)
	time.sleep(1)

search_results = driver.find_elements_by_xpath("//div[@id='links']/div/div/h2/a[@class='result__a']")
#print(len(search_results))

for result in search_results:
  if str(result.get_attribute("href")).endswith('.pdf') == False:
    ulist.append(result.get_attribute("href"))                       
                       
driver.quit()
vlist =[]
ulist = list(set(ulist)-set(vlist))
vlist = list(set(vlist))
urls = ulist + vlist
print('Total pages: %s' % len(urls))

Total pages: 313


In [ ]:
#@title ¡¡¡Venga 100 arañas!!!
#100 threads to store every page html result
@threads(100)
def request_get(url):
  """
  Requests get every url from search results
  """
  try:
    return requests.get(url, headers = headers, timeout=30)

  except requests.exceptions.HTTPError as e:
    print("url HTTPError! url: %s\n" % url)
    pass
  except requests.exceptions.Timeout as e:
    print("url connection timeout! url: %s\n" % url)
    pass 
  except requests.exceptions.TooManyRedirects as e:
    print("url too many redirects! url: %s\n" % url)
    pass
  except requests.exceptions.ConnectionError as e:
    print("url connection failed! url: %s\n" % url)
    pass
  except requests.exceptions.RequestException as e:
    print("url requestexception! url: %s\n" % url)
    pass
  
  

start = time.time()
responses = [[str(url),request_get(url)] for url in urls]
html = [[response[0], BeautifulSoup(response[1].text, 'lxml')] for response in responses if hasattr(response[1],"text")]
end = time.time()
print("Time: %f seconds" % (end - start))


#test all pages if it has match title
for page in html:
  if urlparse(page[0]).netloc == 'www.youtube.com': #enter youtube web
        
        youtube_title = page[1].find('h1', attrs={'class':'watch-title-container'})# find youtube title
        if youtube_title:
          test_set = tokenizer(youtube_title.text.strip())
          
          if len(title_set & test_set) >= (len(title_set)*2)/3:
            rlist.append([youtube_title.text.strip(), page[0], 'YouTube'])
  
  else:
    meta_ogtitle = page[1].find('meta', property="og:title")
    meta_title = page[1].find('meta', property="title")
    headtitle = page[1].find('title')
    fuente = page[1].find('meta', property="og:site_name")
    if meta_ogtitle:
      try:
        test_set = tokenizer(meta_ogtitle["content"])

        if len(title_set & test_set) >= (len(title_set)*2)/3:
          rlist.append([meta_ogtitle["content"], page[0], fuente['content'] if fuente else urlparse(page[0]).netloc])
      except:
        print(str(i)+'meta[content] problem: '+str(page[0]))
        continue
    elif meta_title:
      try:
        test_set = tokenizer(meta_title["content"])

        if len(title_set & test_set) >= (len(title_set)*2)/3:
          rlist.append([meta_title["content"], page[0], fuente['content'] if fuente else urlparse(page[0]).netloc])
      except:
        print(str(i)+'meta[content] problem: '+str(page[0]))
        continue
    elif headtitle:
      try:
        test_set = tokenizer(headtitle.text)

        if len(title_set & test_set) >= (len(title_set)*2)/3:
          rlist.append([headtitle.text, page[0], fuente['content'] if fuente else urlparse(page[0]).netloc])
      except:
        print(str(i)+'title problem: '+str(page[0]))
        continue
    
    else:
      h = page[1].find_all({'h1','h2','h3'}, limit=6)
      for x in h:

        test_set = tokenizer(x.text)

        if len(title_set & test_set) >= (len(title_set)*2)/3:
          rlist.append([x.text, page[0], fuente['content'] if fuente else urlparse(page[0]).netloc])
f'Total impactos: {len(rlist)}'

Time: 19.752707 seconds


'Total impactos: 61'

In [ ]:
#@title Autorización la cuenta de Google
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
#@title Nombre de Sheet

nombre_sheet = 'Test_10072019' #@param {type:"string"}

In [ ]:
#@title NUEVO SHEET
# Creat new sheet in Google Drive
sh = gc.create(nombre_sheet)
worksheet = gc.open(nombre_sheet).sheet1

# Write the results to the sheet
cc=0
cell_ls = worksheet.range(1,1,len(rlist),3)

for c in cell_ls:
  c.value = [x for b in rlist for x in b][cc]
  cc = cc+1
worksheet.update_cells(cell_ls)

{'spreadsheetId': '1Po2opq6G_-q4kcrrkpDsE_NMoDpOIAEAdvz7Tyy6D8k',
 'updatedCells': 183,
 'updatedColumns': 3,
 'updatedRange': 'Sheet1!A1:C61',
 'updatedRows': 61}

In [ ]:
#@title AGRUPACIÓN
# Extract datas from Google sheet to pandas

worksheet = gc.open(nombre_sheet).sheet1
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows)
df = df.drop_duplicates([1],keep='first')
df = df.sort_values([2,0])
df = df.rename(columns={0:'title',1:'url',2:'source'})

# Group by the same source with same title
df_ = df.groupby(['source','title',])['url'].apply(list).reset_index()
df_ = df_[['title', 'source', 'url']]
df_.to_excel('/content/Fuentes_agurapado.xlsx')

rlist = []
rlist = df_.values.tolist()
print('Total impactos: %s'% len(df.index))
print('Total pantallas: %s' % len(df_.index))

Total impactos: 61
Total pantallas: 26


In [ ]:
#@title Captura las pantallas y generar Word_INFORME
# Screenshot
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.implicitly_wait(10)

x=1
for i in rlist:
  try:
    driver.implicitly_wait(10)
    driver.get(i[2][0])
  except: 
    
    pass
  
  time.sleep(2)
  driver.save_screenshot("/content/captura_{0}.png".format(str(x)))
  x += 1
driver.quit()


from PIL import Image
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_ALIGN_VERTICAL, WD_TABLE_ALIGNMENT
from docx.shared import Inches, Cm, Pt, RGBColor
import datetime
import time
from pytz import timezone
import pytz

def time_zone():
  now = datetime.datetime.now()
  start = timezone('Europe/London')
  location = start.localize(now)
  madrid = timezone('Europe/Madrid')
  mad = location.astimezone(madrid)
  time = mad.strftime('%d/%m/%Y')
  return(time)

x = 1
document = Document()

sections = document.sections
#set cover page margin
sections[0].top_margin = Cm(0)
sections[0].bottom_margin = Cm(0)
sections[0].left_margin = Cm(0)
sections[0].right_margin = Cm(0)

#start the rest pages
for i in rlist:
  #document.add_page_break()
  
  #head table efe log with title
  table = document.add_table(rows=1, cols=2, style='Medium List 2 Accent 1')
  table.autofit = False
  table.alignment = WD_TABLE_ALIGNMENT.CENTER

  efelogo = table.rows[0].cells[0] # table[0,0]
  paragraph = efelogo.paragraphs[0]
  efelogo.width = Inches(1.5)
  run = paragraph.add_run()
  run.add_picture('/content/efe_logo.jpg', width=Inches(1.3))
  paragraph.paragraph_format.space_after = Inches(0.2)

  title = table.rows[0].cells[1]
  title.width = Inches(4.7)
  title.text='Informe de Seguimiento:\n'+ informe_de_seguimiento
  title.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
  paragraph = title.paragraphs[0]
  run = paragraph.runs

  font = run[0].font
  font.size = Pt(16)
  font.name = 'Cambria'
  font.bold = True

  document.add_paragraph()
  
  #add second table
  table = document.add_table(rows=3, cols=2, style='Table Grid')
  table.alignment = WD_TABLE_ALIGNMENT.CENTER
  table.autofit = False
  r0 = table.rows[0].cells
  r0[0].text = 'Titulo:'
  r0[1].text = i[0]
  r1 = table.rows[1].cells
  r1[0].text = 'Fuente:'
  r1[1].text = i[1]
  r2 = table.rows[2].cells
  r2[0].text = 'Enlace:'
  
  for count, url in enumerate(i[2]):
  
    print(count, url)
    p = r2[1].paragraphs[count]
    r2[1].add_paragraph()
    p.paragraph_format.space_after = Inches(0.05)
    link = url
    texto = url
    add_hyperlink(p, link, texto, '0000FF')
     
  delete_paragraph(r2[1].paragraphs[-1])

  col_0 = table.columns[0].cells
  for cell in col_0:
    paragraphs = cell.paragraphs
    cell.width = Cm(2.19)
    for paragraph in paragraphs:
      paragraph.paragraph_format.space_before = Inches(0.05)
      paragraph.paragraph_format.space_after = Inches(0.08)
      for run in paragraph.runs:
        font = run.font
        font.size= Pt(14)
        font.name = 'Cambria'
        font.bold = True
        font.color.rgb = RGBColor(54, 95, 145)

  col_1 = table.columns[1].cells
  for cell in col_1:
    cell.paragraphs[0].paragraph_format.space_before = Inches(0.09)
    cell.width = Cm(13.77)
    
    
  
  
  document.add_paragraph()
  
  document.add_picture("/content/captura_{0}.png".format(str(x)), width=Inches(6))
  last_paragraph = document.paragraphs[-1]
  last_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
  document.add_page_break()
  x = x +1

document.save('/content/INFORME_{0}_part.docx'.format(informe_de_seguimiento))

0 https://www.diariomas.hn/2019/07/10/militao-soy-un-jugador-polivalente-que-juega-por-amor-a-lo-que-hace/
0 https://www.elconfidencial.com/ultima-hora-en-vivo/2019-07-10/militao-soy-un-jugador-polivalente-que-juega-por-amor-a-lo-que-hace_2395691/
0 https://www.elgrafico.com/futbol/Militao-Soy-un-jugador-polivalente-que-juega-por-amor-a-lo-que-hace-20190710-0002.html
0 https://eldesmarque.com/madrid/real-madrid/noticias/164088-militao-soy-un-jugador-polivalente-que-juega-por-amor-a-lo-que-hace
0 https://espanol.eurosport.com/futbol/militao-espero-hacer-historia-en-el-real-madrid_sto7365063/story.shtml
0 https://www.eurosport.es/futbol/militao-espero-hacer-historia-en-el-real-madrid_sto7365063/story.shtml
0 https://flipboard.com/@FlipboardEspana/militao-soy-un-jugador-polivalente-que-juega-por-amor-a-lo-que-hace/a-l3k4pTkuTSWc39w_VzX0QA%3Aa%3A130570571-4ef29aefba/eleconomista.es
1 https://flipboard.com/section/sid%2F8536ebsgkeokosfo/militao-soy-un-jugador-polivalente-que-juega-por-amor-

In [ ]:
#@title Twitter
max_tweets = 1000000 #Maximum number of tweets we want to collect
#search_query = 'Militao: "Soy un jugador polivalente que juega por amor a lo que hace"'
since_date = '2019-07-01'

consumer_key = 'bMPhc48pp7riQLSlw3WEL38Ll'
consumer_secret = 'oZEsbpsfHeaGaQtvuqiKEJfh9UdIsB9nGLXvsA9TWMZJwGwfU4'

#application-only authentication allows for 450 queries every 15 minutes
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


if (not api):
  print('Problem connecting to API')
  
twlist = []
c = tweepy.Cursor(api.search, q=query, since=since_date).items(max_tweets)
for tweet in c:
  username = '@'+tweet.user.screen_name
  #print(username)
  #print(tweet.created_at)
  #print(tweet.text)
  tw_url = 'https://twitter.com/'+tweet.user.screen_name+'/status/'+tweet.id_str
  #print(tw_url)
  #print('\n')
  twlist.append([username, tweet.created_at, tweet.text, tw_url, 'Twitter'])
print('Tweets counts:'+ str(len(twlist)))

Tweets counts:22


In [ ]:
df = pd.DataFrame(twlist)
df

,0,1,2,3,4
0,@luisarmando930,2019-07-10 17:08:29,"RT @MonteCarlo930: Militao: ""Soy un jugador po...",https://twitter.com/luisarmando930/status/1149...,Twitter
1,@aideportes,2019-07-10 16:02:11,"Militao: ""Soy un jugador polivalente que juega...",https://twitter.com/aideportes/status/11489857...,Twitter
2,@DiarioMashn,2019-07-10 15:58:11,Militao: “Soy un jugador polivalente que juega...,https://twitter.com/DiarioMashn/status/1148984...,Twitter
3,@Estadio_ED,2019-07-10 15:45:14,"Militao: ""Soy un jugador polivalente que juega...",https://twitter.com/Estadio_ED/status/11489815...,Twitter
4,@andaluciainf,2019-07-10 15:31:41,"Militao: ""Soy un jugador polivalente que juega...",https://twitter.com/andaluciainf/status/114897...,Twitter
5,@avelms,2019-07-10 15:29:41,"RT @RealGomezRM: Militao: ""Soy un jugador poli...",https://twitter.com/avelms/status/114897759807...,Twitter
6,@MonteCarlo930,2019-07-10 14:56:28,"Militao: ""Soy un jugador polivalente que juega...",https://twitter.com/MonteCarlo930/status/11489...,Twitter
7,@Magali_Aly_Mar,2019-07-10 14:43:45,RT @DiarioOpinion: El defensa internacional br...,https://twitter.com/Magali_Aly_Mar/status/1148...,Twitter
8,@sport,2019-07-10 14:41:35,"#RealMadrid ⚪ | Militao: ""Soy un jugador poliv...",https://twitter.com/sport/status/1148965496429...,Twitter
9,@DiarioOpinion,2019-07-10 14:40:10,El defensa internacional brasileño Eder Milita...,https://twitter.com/DiarioOpinion/status/11489...,Twitter


In [ ]:
for i in df[3]:
  print(i)

https://twitter.com/luisarmando930/status/1149002464475209728
https://twitter.com/aideportes/status/1148985777243791360
https://twitter.com/DiarioMashn/status/1148984774242926593
https://twitter.com/Estadio_ED/status/1148981515222769666
https://twitter.com/andaluciainf/status/1148978105207447552
https://twitter.com/avelms/status/1148977598078312453
https://twitter.com/MonteCarlo930/status/1148969242651430912
https://twitter.com/Magali_Aly_Mar/status/1148966040602402816
https://twitter.com/sport/status/1148965496429174784
https://twitter.com/DiarioOpinion/status/1148965139808477186
https://twitter.com/EFEdeportes/status/1148959371940024321
https://twitter.com/RealEspartaB/status/1148958329202515968
https://twitter.com/CJara_6/status/1148956341341446144
https://twitter.com/golestvcom/status/1148953884125065216
https://twitter.com/rmadridsite_/status/1148951766656069633
https://twitter.com/AppRealMadridCF/status/1148951394751254534
https://twitter.com/RealGomezRM/status/114895004902984089

In [ ]:
from docx import Document
from docx.shared import Inches



links = ['https://es-us.noticias.yahoo.com/ejecutiva-bank-of-america-aplaude-tenacidad-empresarios-hispanos-170800851--sector.html',
         'http://elminnesotadehoy.com/ejecutiva-de-bank-of-america-aplaude-la-tenacidad-de-empresarios-hispanos/',
         'http://snewsi.com/id/19187510236/Ejecutiva-de-Bank-of-America-aplaude-la-tenacidad-de-empresarios-hispanos']

document = Document()

footer = document.sections[0].footer
footer.text = 'hola'
table = document.add_table(rows=3, cols=2, style='Table Grid')
r1 = table.rows[1].cells
p0 = r1[1].paragraphs[0]
add_hyperlink(p0, 'https://www.google.com/', 'https://www.google.com/', '0000FF')
r2 = table.rows[2].cells
r2[0].text = 'Enlace:'

for count, url in enumerate(links):
  
  print(count, url)
  p = r2[1].paragraphs[count]
  r2[1].add_paragraph()
  p.paragraph_format.space_after = Inches(0.05)
  link = url
  texto = url
  add_hyperlink(p, link, texto, '0000FF')
     
delete_paragraph(r2[1].paragraphs[-1])
document.save('/content/test.docx')



In [ ]:


diariovasco = ['https://www.diariovasco.com/agencias/201809/15/mireia-belmonte-participa-travesia-1258045.html']

x=1
for i in diariovasco:
  driver = webdriver.Chrome('chromedriver',options=chrome_options)
  driver.implicitly_wait(10)
  try:
    
    driver.get(i)
  except: 
    pass
  
  time.sleep(2)
  driver.save_screenshot("/content/captura_vasco_{0}.png".format(str(x)))
  x += 1
  driver.quit()

In [ ]:
#Test for value counts
worksheet = gc.open(nombre_file).sheet1
rows = worksheet.get_all_values()

import pandas as pd
df = pd.DataFrame.from_records(rows)
df[0].value_counts()

In [ ]:
import numpy as np

# Open our new sheet and read some data.
worksheet_clients = gc.open('clientes_urls_list').sheet1

# get_all_values gives a list of rows.
clients_list = worksheet_clients.get_all_values()

df['client'] = np.nan

for index, row in df.iterrows():
  domain = urlparse(str(row['url'])).netloc
  for c in clients_list:
    if str(domain) in c:
      df.loc[index, 'client'] = 'Si'
      break
    else:
      df.loc[index, 'client'] = 'No'
df

In [ ]:
#final list to googlesheet
flist = df_[[0,1,2]].values.tolist()
nombre_file = nombre_file+'final'
sh = gc.create(nombre_file)

worksheet = gc.open(nombre_file).sheet1

cc=0

cell_ls = worksheet.range(1,1,len(flist),3)

for c in cell_ls:
  c.value = [x for b in flist for x in b][cc]
  cc = cc+1
worksheet.update_cells(cell_ls)